In [1]:
#
#
#

#The dataset availability and their size is a huge issue in DL and ML. So, the code checks if the GAN generated dataset is fiseabile to be used as training datset.

#The main purpose of this code is to check the efficiency of GAN generated artificial data in classification.
#The GAN artificial dataset is generated using CGAN network.

#The original MNIST-Fashion Dataset is first classified and the accuracy is obtained.
#Next, the original dataset and the artificial dataset are combined in 50% proportions and tested again for classification using CNN.

#The hyperparameters like epoch, sample_interval, batch_size, learning rate (alpha) are changed continuously and the current combination provides
#with higher accuracy for 50% combined data, but it is an minimal increase.

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.layers import (
        Activation, BatchNormalization, Concatenate, Dense, Dropout, Multiply,
        Embedding, Flatten, Input, Reshape, LeakyReLU, Conv2D, Conv2DTranspose, MaxPooling2D) 
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model

# Define important parameters
img_shape = (28,28,1)
z_dim =100
n_class= 10

In [2]:
def generator_model(z_dim):

    model = Sequential()

    model.add(Dense(256 * 7 * 7, input_dim=z_dim,))
    model.add(Reshape((7, 7, 256)))

    model.add(Conv2DTranspose(128, 3, 2, padding='same',))
    model.add(LeakyReLU(alpha =0.01))

    model.add(Conv2DTranspose(64, 3, 1, padding='same',))
    model.add(LeakyReLU(alpha =0.01))

    model.add(Conv2DTranspose(1, 3, 2, padding='same',))
    model.add(LeakyReLU(alpha =0.01))

    return model

# generator input 
def generator(z_dim):
    # latent input
    z = Input(shape=(z_dim, ))
    # label input
    label = Input(shape=(1, ), dtype='int32')
    # convert label to embedding
    label_embedding = Embedding(n_class, z_dim)(label)

    label_embedding = Flatten()(label_embedding)
    # dot product two inputs
    joined_representation = Multiply()([z, label_embedding])

    generator = generator_model(z_dim)

    conditioned_img = generator(joined_representation)

    model =  Model([z, label], conditioned_img)
    # save model blueprint to image
    plot_model(model,'generator.jpg',show_shapes=True,show_dtype=True)

    return model

In [3]:
# discriminator CNN model
def discriminator_model(img_shape):

    model = Sequential()

    model.add(Conv2D(64,3,2,input_shape=(img_shape[0], img_shape[1], img_shape[2] + 1),))
    model.add(LeakyReLU(alpha =0.01))

    model.add(Conv2D(64,3,2,input_shape=img_shape,padding='same',))
    model.add(LeakyReLU(alpha =0.001))

    model.add(Conv2D(128,3,2,input_shape=img_shape,padding='same',))
    model.add(LeakyReLU(alpha =0.001))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    return model


def discriminator(img_shape):
    # image input
    img = Input(shape=img_shape)
    # label input
    label = Input(shape=(1, ), dtype='int32')

    label_embedding = Embedding(n_class, np.prod(img_shape),input_length=1)(label)

    label_embedding = Flatten()(label_embedding)

    label_embedding = Reshape(img_shape)(label_embedding)
    # concatenate the image and label
    concatenated = Concatenate(axis=-1)([img, label_embedding])

    discriminator = discriminator_model(img_shape)

    classification = discriminator(concatenated)

    model = Model([img, label], classification)

    plot_model(model,'discriminator.jpg',show_shapes=True,show_dtype=True)

    return model

In [4]:
# define a complete GAN architecture
def cgan(generator, discriminator):

    z = Input(shape=(z_dim, ))

    label = Input(shape=(1, ))

    img = generator([z, label])

    classification = discriminator([img, label])

    model = Model([z, label], classification)
    
    return model

discriminator = discriminator(img_shape)
# compile the discriminator architecture 
discriminator.compile(loss='binary_crossentropy',
                      optimizer=Adam(),
                      metrics=['accuracy'])

generator = generator(z_dim)
# set discriminator to non-trainanle 
discriminator.trainable = False
# compile the whole C-GAN architectu
cgan = cgan(generator, discriminator)
cgan.compile(loss='binary_crossentropy', optimizer=Adam())

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [5]:
# label to category dictionary
dict_clothes ={0: "T-shirt/top",1: "Trouser",2: "Pullover",
               3: "Dress",4: "Coat",5: "Sandal",6: "Shirt",
               7: "Sneaker",8: "Bag",9: "Ankle boot"}

# function to plot and save sample images
def plot_sample_images(epoch ,rows=5,columns=4):

    z = np.random.normal(0, 1, (rows * columns, z_dim))
    a =np.arange(0,10)
    b =np.arange(0,10)

    labels = np.append(a,b).reshape(-1,1)
    
    gen_imgs = generator.predict([z, labels])

    gen_imgs = 0.5 * gen_imgs + 0.5
    print("Epoch : %d "%(epoch+1))
    fig, axs = plt.subplots(rows,
                            columns,
                            figsize =(50, 20),
                            sharey=True,
                            sharex=True)

    cnt = 0
    for i in range(rows):
        for j in range(columns):
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            axs[i, j].set_title("Type: %s" % dict_clothes.get(labels[cnt][0]))
            cnt += 1
    fig.savefig('image%d.jpg'%(epoch))

 
# define training step
def train(epochs, batch_size, sample_interval):
    #  import Fashion-MNIST dataset
    (x_train, y_train), (x_test, y_test)  = fashion_mnist.load_data()
    X_train = np.concatenate([x_train, x_test])
    Y_train = np.concatenate([y_train, y_test])

    X_train = X_train.astype("float32") / 255.0
    X_train = np.reshape(X_train, (-1, 28, 28, 1))


    real = np.ones((batch_size, 1))

    fake = np.zeros((batch_size, 1))
    
    for epoch in range(epochs): 
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs, labels = X_train[idx], Y_train[idx]

        z = np.random.normal(0, 1, (batch_size, z_dim))
        # generate images from generator
        gen_imgs = generator.predict([z, labels])
        # pass real an generated images to the discriminator and ctrain on them
        d_loss_real = discriminator.train_on_batch([imgs, labels], real)
        d_loss_fake = discriminator.train_on_batch([gen_imgs, labels], fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        z = np.random.normal(0, 1, (batch_size, z_dim))

        labels = np.random.randint(0, n_class, batch_size).reshape(-1, 1)
   
        g_loss = cgan.train_on_batch([z, labels], real)
        
        gen_imgs.to_csv('artificial data.csv',mode='a')
        
        if (epoch + 1) % sample_interval == 0:

            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" %(epoch + 1, d_loss[0], 100 * d_loss[1], g_loss))

            plot_sample_images(epoch+1)
iterations = 2000
batch_size = 128
sample_interval = 2000

train(iterations, batch_size, sample_interval)

4/4 [==============================] - 0s 33ms/step


In [6]:
#Loading the Training Dataset.
x=pd.read_csv(r"C:\Users\Surya\Desktop\MNIST\fashion-mnist_train.csv")

#Labels.
y_train=x['label']

#Features.
x_train=x.drop(['label'],axis=1)

#Storing the Features, for further need.
x_train0=x_train[2::2]
y_train0=y_train[2::2]

#Reshaping into a 3D numpy array to apply CNN.
x_train=np.asarray(x_train)
x_train=x_train.reshape([-1,28,28,1])/255

In [7]:
#Loading the Testing Dataset.
x=pd.read_csv(r"C:\Users\Surya\Desktop\MNIST\fashion-mnist_test.csv")

#Labels.
y_test=x['label']

#Features.
x_test=x.drop(['label'],axis=1)

#Reshaping into a 3D numpy array to apply CNN.
x_test=np.asarray(x_test)
x_test=x_test.reshape([-1,28,28,1])/255

In [8]:
#CNN model with relu activation functions and sigmoid at output layer.
model = Sequential()
model.add(Conv2D(filters=64, kernel_size=(3,3), input_shape=(28,28,1), activation='relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(rate=0.5))
model.add(Flatten())
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=10, activation='sigmoid'))

In [9]:
#Complete model is now compiled and the description is shown.
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 26, 26, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 13, 13, 64)        0         
                                                                 
 flatten_3 (Flatten)         (None, 10816)             0         
                                                                 
 dense_2 (Dense)             (None, 16)                173072    
                                                                 
 dense_3 (Dense)             (None, 10)                170       
                                                      

In [10]:
#Fitting the model with the original MNIST-Fashion Dataset using CNN.
model.fit(x=x_train, y=y_train, batch_size=512, epochs=5, validation_data=(x_test, y_test))

Epoch 1/5
118/118 [==============================] - 37s 304ms/step - loss: 0.7222 - accuracy: 0.7588 - val_loss: 0.4579 - val_accuracy: 0.8412
Epoch 2/5
118/118 [==============================] - 35s 298ms/step - loss: 0.4435 - accuracy: 0.8420 - val_loss: 0.3945 - val_accuracy: 0.8641
Epoch 3/5
118/118 [==============================] - 35s 295ms/step - loss: 0.3930 - accuracy: 0.8620 - val_loss: 0.3615 - val_accuracy: 0.8777
Epoch 4/5
118/118 [==============================] - 35s 294ms/step - loss: 0.3629 - accuracy: 0.8727 - val_loss: 0.3527 - val_accuracy: 0.8775
Epoch 5/5
118/118 [==============================] - 35s 301ms/step - loss: 0.3459 - accuracy: 0.8788 - val_loss: 0.3171 - val_accuracy: 0.8925


In [11]:
#Accuracy of original MNIST-Fashion Dataset using CNN.
accuracy=model.evaluate(x_test, y_test)

313/313 [==============================] - 3s 8ms/step - loss: 0.3171 - accuracy: 0.8925


In [20]:
#The artificial dataset generated using GAN.
x=pd.read_csv(r"C:\Users\Surya\Desktop\MNIST\artificial data.csv")

#Labels are available as CGAN is implemented. 
y_train1=x['label']

#Features.
x_train1=x.drop(['label'],axis=1)

#Reshaping.
x_train1=np.asarray(x_train1)
x_train1=x_train1.reshape([-1,28,28,1])/255

In [29]:
#Half of the actual MNIST- Fashion Dataset.
x_train1=x_train0
y_train1=y_train0

#New dataset with 50% original and 50% CGAN generated is used.
x_train=np.concatenate((x_train1,x_train0),axis=0)
y_train=np.concatenate((y_train1,y_train0),axis=0)

x_train=x_train.astype(float)
y_train=y_train.astype(float)

In [30]:
#Fitting the model.
model.fit(x=x_train, y=y_train, batch_size=512, epochs=5, validation_data=(x_test, y_test))

Epoch 1/5
235/235 [==============================] - 72s 302ms/step - loss: 0.3270 - accuracy: 0.8842 - val_loss: 0.2973 - val_accuracy: 0.8990
Epoch 2/5
235/235 [==============================] - 74s 316ms/step - loss: 0.3050 - accuracy: 0.8927 - val_loss: 0.2887 - val_accuracy: 0.8980
Epoch 3/5
235/235 [==============================] - 70s 296ms/step - loss: 0.2893 - accuracy: 0.8974 - val_loss: 0.2796 - val_accuracy: 0.9029
Epoch 4/5
235/235 [==============================] - 69s 292ms/step - loss: 0.2774 - accuracy: 0.9011 - val_loss: 0.2683 - val_accuracy: 0.9065
Epoch 5/5
235/235 [==============================] - 69s 295ms/step - loss: 0.2697 - accuracy: 0.9040 - val_loss: 0.2601 - val_accuracy: 0.9100


In [31]:
#Accuracy.
accuracy=model.evaluate(x_test, y_test)

313/313 [==============================] - 4s 12ms/step - loss: 0.2601 - accuracy: 0.9100
